# Demo: refactored implementation of $\Omega$

This notebook gives an example of the new implementation of $\Omega$. The aim of the implementation has so far been to make as few alterations as possible to the existing codebase, while still enabling the user considerably more flexibility in the definition of $\Omega$. 

## Math Assumptions

The primary mathematical assumption behind the new implementation is that $\Omega(z)$ is always expressible as a function of the partition vector of $z$. This is the same assumption that we were using throughout our previous version. The aim of the new implementation is that special cases can be handled with significantly more peformant code. 

In [1]:
using Pkg; Pkg.activate(".")
using HypergraphModularity

using StatsBase

 Activating environment at `~/hypergraph_modularities_code/Project.toml`


In [ ]:
n = 20
Z = rand(1:5, n)
ϑ = dropdims(ones(1,n) + rand(1,n), dims = 1)
μ = mean(ϑ)
kmax = 4;

## The `IntensityFunction` struct

Now we're ready to construct $\Omega$. In the new idiom, $\Omega$ is an `IntensityFunction`, which contains four fields. 
Let's look at two examples, which are both implemented in `src/omega.jl`. 

```julia
function partitionIntensityFunction(ω, kmax)
    range      = partitionsUpTo(kmax)
    P          = partitionize
    aggregator = identity
    return IntensityFunction(ω, P, range, aggregator)
end
```

This `IntensityFunction` corresponds to the highly general `partitionize`-based framework we were using previously. Here are its fields: 

<dl>
    <dt> ω </dt> <dd> A parameterized function whose valid inputs are the elements of `range`. </dd>
    <dt> <code>P</code> </dt> <dd> A function that maps group label vectors (i.e. subvectors of $Z$) to <i>feature</i> vectors. <code>partitionize()</code> is an example, as is the polyadic $\delta$-function. </dd>
    <dt> <code>range</code> </dt> <dd> The set of all possible values of <code>P</code>. Also assumed to be the domain of the function $\omega$. This nomenclature is perhaps confusing and may be revised.</dd>
    <dt> <code>aggregator</code> </dt> <dd> A function that maps partition vectors (of the kind returned by <code>partitionize()</code> to elements of <code>range</code>. This is included for technical purposes related to the calculation of the volume term in modularity. It would be desirable to deprecate it if possible.  </dd>
</dl>

So, the code above creates an intensity function that operates on subvectors of $Z$ by first `partitionize()`ing them and then applying the function $\omega$. No aggregation is needed.  

On the other hand, here's an alternative that corresponds to the all-or-nothing cut. In this case, the feature map $P$ computes two entries: whether or not all entries of $z$ are the same, and the length of $z$. The `range` gives all possible results (up to size `kmax`). The aggregator takes a partition vector and returns a feature. The specified function $\omega$ should then operate on the features. 

```julia
function allOrNothingIntensityFunction(ω, kmax)
    range      = [(1.0*x, y) for x = 0:1 for y = 1:kmax]
    P          = z->(all(z[1] .== z), length(z))
    aggregator = p->(length(p) == 1, sum(p))
    return IntensityFunction(ω, P, range, aggregator)
end
```
Ok, let's try it out: 



In [ ]:
function ω(p, α)
    k = sum(p)
    return sum(p)/sum((p .* (1:length(p)).^α[k])) / n^(α[kmax+k]*k)
end

α = vcat(repeat([5.0], kmax), 0.2*(1:kmax))
Ω = partitionIntensityFunction(ω, kmax);

typeof(Ω)

In [ ]:
H = sampleSBM(Z, ϑ, Ω;α=α, kmax=kmax, kmin = 1)

# TODO: check on inference and algorithms

# CONJECTURE: 

we have introduced a bug related to CutDiff_OneNode that is causing issues. Consider whether we should try to squash the bug or reimplement SuperNodeLouvain using more familiar functions such as cutDiff(). Major question will be whether this introduces additional expense (i.e. whether we gain from the penalty term used in the current code). 

I **think** that we do gain from storying the penalties -- should cut the number of calls to `Omega.P` by about half, which is certainly desirable. 




In [ ]:
dataset = "contact-primary-school"
kmax_ = 6

H, Z = read_hypergraph_data(dataset,kmax_)


kmax = maximum(keys(H.E))
kmin = minimum(keys(H.E))


n = length(H.D)

# full, partition-based intensity function

function ω(p, α)
    k = sum(p)
    return sum(p)/sum((p .* (1:length(p)).^α[k])) / n^(α[kmax+k]*k)
end

Ω = partitionIntensityFunction(ω, kmax);

α = zeros(2*kmax)

# ---

# all-or-nothing
# issue: no nodes move clusters with this intensity function
# function ω(p,α)
#     k = p[2]
#     δ = p[1]
#     return (1.0*n)^(α[k + (1-δ)*kmax]*k)
# end

# Ω = allOrNothingIntensityFunction(ω, kmax);

In [ ]:
Z_dyadic = CliqueExpansionModularity(H);

In [ ]:
α, f = learnParameters(H, Z, Ω, α; ftol_abs = 1) # seems to still fail for all-or-nothing

In [ ]:
Z = HypergraphModularity.HyperLouvain_(H, kmax, Ω; α=α, verbose = true);

println(length(unique(Z)))

In [ ]:
Z = HypergraphModularity.HyperLouvain(H, kmax, Ω; α=α, verbose = true);
println(length(unique(Z)))

In [ ]:
for i = 1:10
    Z = SuperNodeLouvain(H, kmax, Ω; α=α, verbose = false, scan_order = "random");
    α, f = learnParameters(H, Z, Ω, α; ftol_abs = 1)
    println(length(unique(Z)), " ", f)
end

In [ ]:
# timing is comparable to previous code, as expected
@time Z = SuperNodeLouvain(H, kmax, Ω; α=α, verbose = false, scan_order = "random");